Estructura que vamos a implementar

Generador de posiciones de vehiculos
Revisar la función Treading.Thread, genera la misma función de python varias veces a la vez

In [ ]:
# Aqui deberíamos poner el generador que tenemos, como idea, podemos guardar los mensajes en un bucket o mandarlos a pupsug, 
# revisar la función que genera varios posiciones random de diferentes vehículos como en el postwork de Javi.
# En el trabajo final, esto lo puede hacer un VM

Generador de posiciones de individuos
Revisar la función Treading.Thread, genera la misma función de python varias veces a la vez

In [ ]:
# Hacer otro generador, que tomando las posiciones de los kml´s y añadiendo  +/- x puntos, nos de posiciones 
# random de los usuarios. Podemos tambien guardarlos en otro bucket para posteriormente leerlos de ahí o mandarlos a pupsug.
# En el trabajo final, esto lo puede hacer un VM 

Colas de mensajes. Tendremos así dos colas de PupSub que leerán los mensajes de los dos buckets: vehículos y clientes

In [ ]:
        """ Part 01: Read data from PubSub. """

        data1 = (
            p
                | "Read From PubSub" >> # leemos del bucket de vehiculos
                | "Parse JSON messages" >> #Tenemos que ver cual es la estructura del mensaje y transformarlo a nuetra conveniencia
        )

        data2 = (
            p
                | "Read From PubSub" >> # leemos del bucket de clientes
                | "Parse JSON messages" >> #Tenemos que ver cual es la estructura del mensaje y transformarlo a nuetra conveniencia
        )

        """ Part 02: Get the aggregated data of the vehicle within the section. """

        data1, data2, processed_data = (
            
             
                | "tenemos que leer los datos de ambos topics" >> #ToDo: Complete this section
                | "Cramos una ventana temporal donde" >> # Tendremos las posiciones de los vehiculos y de los clientes
                | "Creamos una funcion (Dofn)" >> # Donde comparemos las distancias entre ambos
                # si esta distancia es > que x match, sino, no match.
                | "Tendremos que guardar los match en bigquery" >> 
                |"Ver donde guardamos los no match si en bigquery o en otro bucket sin más"
        
    

Leeremos de bigquery los resultados match y los mostramos con streamlit.

en local

In [12]:
# Generador de vehiculos local

import os
import xml.etree.ElementTree as ET
import random
import time


def read_kml(oferta, file_path):
    data = {"id_oferta": [], "punto": [], "latitude": [], "longitude": [], "Coordinates":[]}
    datos_longitude = []
    datos_latitude = []

    with open(file_path, "r", encoding="utf-8") as file:
        kml_data = file.read()

    root = ET.fromstring(kml_data)
    coords = root.find(".//{http://www.opengis.net/kml/2.2}LineString/{http://www.opengis.net/kml/2.2}coordinates")

    if coords is not None:
        coords_str = coords.text
        coords_list = [tuple(map(float, _.split(','))) for _ in coords_str.split()]

        for i, coords in enumerate(coords_list):
            data["id_oferta"].append(oferta)
            data["punto"].append(i + 1)
            data["latitude"].append(coords[1])
            data["longitude"].append(coords[0])
            datos_latitude.append(coords[1])
            datos_longitude.append(coords[0])
            data["coordinates"]=""
            
            data["coordinates"].append((coords[1], coords[0]))

    return datos_longitude, datos_latitude


def get_coords_finales(directory):
    latitudes_finales = []
    longitudes_finales = []

    for filename in os.listdir(directory):
        if filename.endswith(".kml"):
            file_path = os.path.join(directory, filename)

            with open(file_path, "r", encoding="utf-8") as archivo:
                datos_kml = archivo.read()

            raiz = ET.fromstring(datos_kml)
            coordenadas = raiz.findall(".//{http://www.opengis.net/kml/2.2}coordinates")

            if coordenadas:
                for coordenada_str in coordenadas:
                    coords_lista = [tuple(map(float, _.split(','))) for _ in coordenada_str.text.split()]
                    for coords in coords_lista:
                        longitudes_finales.append(coords[0])
                        latitudes_finales.append(coords[1])

    return latitudes_finales, longitudes_finales


def gen_ofertas(num_ofertas, directory):
    datos_longitude_total = []
    datos_latitude_total = []

    for i in range(1, num_ofertas + 1):
        file_id = random.randint(1, 27)
        file_path = os.path.join(directory, f"{file_id}.kml")
        datos_longitude, datos_latitude = read_kml(oferta=i, file_path=file_path)
        datos_longitude_total.extend(datos_longitude)
        datos_latitude_total.extend(datos_latitude)

    return datos_longitude_total, datos_latitude_total

num_ofertas = 3  # Número de ofertas que deseas generar
directory = "C:/Users/josan/Documents/GitHub/DATA_PROJECT_2/Ejecutable/coordenadas"  # Directorio que contiene los archivos KML

# Generar las ofertas y obtener las longitudes y latitudes
longitudes, latitudes = gen_ofertas(num_ofertas, directory)

# Crear el mensaje completo
mensaje=({"id_oferta": "", "punto": "", "latitude": "", "longitude": ""})
for i in range(len(longitudes)):
    oferta_id = (i // num_ofertas) + 1
    punto = (i % num_ofertas) + 1
    latitude = latitudes[i]
    longitude = longitudes[i]
    mensaje=({"id_oferta": oferta_id, "punto": punto, "latitude": latitude, "longitude": longitude})
    print(mensaje)
    time.sleep(2)  # Esperar dos segundos antes de generar el próximo mensaje


{'id_oferta': 1, 'punto': 1, 'latitude': 39.46242, 'longitude': -0.32912}
{'id_oferta': 1, 'punto': 2, 'latitude': 39.46242, 'longitude': -0.32918}
{'id_oferta': 1, 'punto': 3, 'latitude': 39.4624, 'longitude': -0.32933}
{'id_oferta': 2, 'punto': 1, 'latitude': 39.46238, 'longitude': -0.32941}
{'id_oferta': 2, 'punto': 2, 'latitude': 39.46238, 'longitude': -0.32943}
{'id_oferta': 2, 'punto': 3, 'latitude': 39.46236, 'longitude': -0.32952}
{'id_oferta': 3, 'punto': 1, 'latitude': 39.46234, 'longitude': -0.32961}
{'id_oferta': 3, 'punto': 2, 'latitude': 39.46232, 'longitude': -0.32971}
{'id_oferta': 3, 'punto': 3, 'latitude': 39.46225, 'longitude': -0.32994}


KeyboardInterrupt: 

In [ ]:
# generacion de solicitudes de Vehiculos local:
def gen_demanda(num_ofertas, directory):
    datos_longitude_total = []
    datos_latitude_total = []

    for i in range(1, num_ofertas + 1):
        file_id = random.randint(1, 27)
        file_path = os.path.join(directory, f"{file_id}.kml")
        datos_longitude, datos_latitude = read_kml(oferta=i, file_path=file_path)
        datos_longitude_total.extend(datos_longitude)
        datos_latitude_total.extend(datos_latitude)

    return datos_longitude_total, datos_latitude_total

num_ofertas = 3  # Número de ofertas que deseas generar
directory = "C:/Users/josan/Documents/GitHub/DATA_PROJECT_2/Ejecutable/coordenadas"  # Directorio que contiene los archivos KML

# Generar las ofertas y obtener las longitudes y latitudes
longitudes, latitudes = gen_demanda(num_ofertas, directory)

# Crear el mensaje completo
mensaje=({"id_demanda": "", "punto": "", "latitude": "", "longitude": ""})
for i in range(len(longitudes)):
    oferta_id = (i // num_ofertas) + 1
    punto = (i % num_ofertas) + 1
    latitude = latitudes[i]
    longitude = longitudes[i]
    mensaje=({"id_demanda": oferta_id, "punto": punto, "latitude": latitude, "longitude": longitude})
    print(mensaje)
    time.sleep(2)  # Esperar dos segundos antes de generar el próximo mensaje

{'id_demanda': 1, 'punto': 1, 'latitude': 39.49943, 'longitude': -0.40782}
{'id_demanda': 1, 'punto': 2, 'latitude': 39.49944, 'longitude': -0.40782}
{'id_demanda': 1, 'punto': 3, 'latitude': 39.49946, 'longitude': -0.40781}
{'id_demanda': 2, 'punto': 1, 'latitude': 39.49947, 'longitude': -0.40781}
{'id_demanda': 2, 'punto': 2, 'latitude': 39.49948, 'longitude': -0.4078}
{'id_demanda': 2, 'punto': 3, 'latitude': 39.49949, 'longitude': -0.40781}
{'id_demanda': 3, 'punto': 1, 'latitude': 39.4995, 'longitude': -0.40781}
{'id_demanda': 3, 'punto': 2, 'latitude': 39.49951, 'longitude': -0.40781}
{'id_demanda': 3, 'punto': 3, 'latitude': 39.49952, 'longitude': -0.40782}
{'id_demanda': 4, 'punto': 1, 'latitude': 39.49953, 'longitude': -0.40783}
{'id_demanda': 4, 'punto': 2, 'latitude': 39.49954, 'longitude': -0.40783}
{'id_demanda': 4, 'punto': 3, 'latitude': 39.49954, 'longitude': -0.40784}
{'id_demanda': 5, 'punto': 1, 'latitude': 39.49954, 'longitude': -0.40785}
{'id_demanda': 5, 'punto': 

KeyboardInterrupt: 

In [13]:
from flask import Flask, request, jsonify
import os
import random
import json
import time
import google.cloud
from google.cloud import pubsub_v1
from google.cloud import storage
from google.cloud.storage import Blob
import xml.etree.ElementTree as ET
import argparse
import logging

Creamos un pipeline de vehiculos:

In [28]:
!pip show apache_beam

Name: apache-beam
Version: 2.51.0
Summary: Apache Beam SDK for Python
Home-page: https://beam.apache.org
Author: Apache Software Foundation
Author-email: dev@beam.apache.org
License: Apache License, Version 2.0
Location: c:\users\josan\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\site-packages
Requires: cloudpickle, crcmod, dill, fastavro, fasteners, grpcio, hdfs, httplib2, js2py, numpy, objsize, orjson, packaging, proto-plus, protobuf, pyarrow, pydot, pymongo, python-dateutil, pytz, regex, requests, typing-extensions, zstandard
Required-by: 


In [29]:
# Import Python Libraries
import logging
import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

In [9]:
from datetime import datetime

datos_demandas = get_coords_finalesClientes("C:/Users/josan/Documents/GitHub/DATA_PROJECT_2/Ejecutable/coordenadas")
datos_ofertas = get_coords_finalesVehiculos("C:/Users/josan/Documents/GitHub/DATA_PROJECT_2/Ejecutable/coordenadas")

class TransformarMensaje(beam.DoFn):
    def process(self, element):
        # Extraer la latitud y longitud del elemento
        latitude = element['latitude']
        longitude = element['longitude']

        # Crear la tupla de dos elementos
        tupla_lat_long = (latitude, longitude)

        # Devolver el mensaje transformado
        return [(tupla_lat_long, element)]



# Crear un pipeline de Apache Beam
with beam.Pipeline(InteractiveRunner()) as p:
    # Leer los datos_ofertas usando la transformación Create
    ofertas_vehiculos= ( 
        p   
        | "Create PCollection" >> beam.Create(datos_ofertas)# datos
        | "Transformamos los datos con el Dofn">> beam.ParDo(TransformarMensaje())
        # | "Agrupamos por clave" >> beam.GroupByKey() no
        # Agrupamos por id vehiculo y por posición
        
    )
    demandas_vehiculos= ( 
        p   
        | "Create PCollection2" >> beam.Create(datos_demandas)# datos
        | "Transformamos los datos con el Dofn2" >> beam.ParDo(TransformarMensaje())
        # | "Agrupamos por clave2" >> beam.GroupByKey()
        
    )
    combined_pcollection = (
        (ofertas_vehiculos, demandas_vehiculos)
        | "Creamos una tercera Pcollection a partir de las anteriores">> beam.CoGroupByKey()
        | "Agrupamos por key" >> beam.GroupByKey() 
        | "Imprimimos" >>beam.Map(print) 
        )


#Agrupar mensaje vehículo por id y comparar sus posiciones con posición vehiculo
   

        





    

KeyboardInterrupt: 

Con el cogroup by key agrupamos todos los datos de los dos subconjuntos en en una lista de datos que tienen la misma clave.
Tengo que probar tambien:
 combined_pcollection = (ofertas_vehiculos, demandas_vehiculos) | beam.CombinePerKey(lambda x, y: (sum(x), sum(y)))
Y si no nos genera el resultado esperado, intentar crar un Dofn que nos dé el resultado que esperamos.

